In [1]:
import spacy
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from spacytextblob.spacytextblob import SpacyTextBlob
from ast import literal_eval

In [2]:
df = pd.read_csv('C:/Users/qsavy_1/CCD/ccd_meta/video_data_final.csv')
df = df[['channelTitle','tags', 'viewCount', 'publishedAt','pushblishDayName']]
df

,channelTitle,tags,viewCount,publishedAt,pushblishDayName
0,ElTrollino,['roblox'],819935.0,2022-07-28T18:15:03Z,NaN
1,ElTrollino,['minecraft'],3900502.0,2022-06-18T20:45:18Z,NaN
2,ElTrollino,['escondite'],2827316.0,2022-06-10T18:54:53Z,NaN
3,ElTrollino,['roblox'],2692418.0,2022-06-04T18:20:42Z,NaN
4,ElTrollino,['minecraft'],4171162.0,2022-05-28T18:00:16Z,NaN
...,...,...,...,...,...
2378,German,NaN,109804.0,2022-01-05T04:30:09Z,NaN
2379,German,"['german garmendia', 'musica', 'pop', 'pop hit...",2040533.0,2020-12-17T21:00:10Z,NaN
2380,German,"['german garmendia', 'musica', 'canciones', 'p...",5487331.0,2020-09-23T21:00:10Z,NaN
2381,German,"['german garmendia', 'german', 'garmendia', 'm...",6483164.0,2020-03-31T22:00:10Z,NaN


In [3]:
tag_class = df['tags'].astype (str)
tag_list = tag_class.values.tolist()
tag_str = ''.join(tag_list)
tag_clean = re.sub(pattern = '[...]',
repl = "",
string = tag_str)


In [4]:
from spacy.matcher import Matcher
nlp = spacy.load('es_core_news_md')

matcher = Matcher(nlp.vocab)
pattern = [
          {'ORTH': "'"},
          {'POS':'PROPN', 'OP':'+'},
          {'ORTH': "'"},
          ] 
matcher.add('PEOPER_NOUNS', [pattern], greedy='LONGEST')
doc_tags = nlp(tag_clean)
matches = matcher(doc_tags)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:10]:
    print(match, doc_tags[match[1]:match[2]])

8022
(18135987400277701921, 53, 57) 'pk xd'
(18135987400277701921, 66, 69) 'mikecrack'
(18135987400277701921, 90, 93) 'mikecrack'
(18135987400277701921, 106, 109) 'Reto'
(18135987400277701921, 110, 113) 'Challenge'
(18135987400277701921, 166, 170) 'pk xd'
(18135987400277701921, 223, 226) 'mikecrack'
(18135987400277701921, 256, 259) 'mikecrack'
(18135987400277701921, 269, 272) 'mikecrack'
(18135987400277701921, 1839, 1843) 'jason voorhees'


In [5]:
channels = df.channelTitle.tolist()

In [6]:
nlp = spacy.blank('en')
ruler = nlp.add_pipe('entity_ruler')
patterns =[]
for tag in doc_tags:
    patterns.append({'label': 'video_tag', 'pattern': tag})
for channel in channels:
    patterns.append({'label': 'YT_channel', 'pattern':channel})
ruler.add_patterns(patterns)

In [7]:
print(doc_tags[:10])

['roblox']['minecraft']['escondite']['roblox']['minecraft']['escondite']['los compas', 'eltrollino',


In [8]:
df_comments = pd.read_csv('C:/Users/qsavy_1/CCD/ccd_meta/comments_data_final.csv')
df_comments.head()

,video_id,comments,comments_no_stopwords
0,uCqLtZhf8xw,['Trollino por fin azes vídeos te echaba de me...,NaN
1,xkQKH6-uNyE,['Me gusta que todos nos unimos en comunidad!'...,NaN
2,MWNpBHATT-w,['Cuando me piden comida yo voy a decir: ¡Trol...,NaN
3,6Qg_FpyPY-Y,"['Si', 'Trollino siempre te alegra el día 💛', ...",NaN
4,IgNj4zJ-Tv0,"['Cierto jugar con us tedes', 'Tú videos me gu...",NaN


In [9]:
df_comments.shape

(66273, 3)

In [11]:
# get 1%
sample = df_comments.sample(frac=0.01)

In [12]:
comments = (sample['comments'].astype (str))
comments_list = comments.values.tolist()
all_words_str = ','.join(comments_list)
all_words_str

'[\'Luisito viaja al monte Everest\', \'Cuando Luisito era humilde.\', \'5:46 la calaverita literaria de bvochido y luisito\', \'El de fantasmas\', \'Amigo te voy a decir algo me encantan tus vídeos pero no tienes buen gusto para adornarlo\', \'A\', \'Bolivia\', \'2021.....\', \'El dinero te lo bastante a qu\', \'Viendo este vidio en pleno 2021 🥺🥺❤\\nLuisillo el diseñador de Bvochido 💖\'],[],[\'Tribunal ya basta de pisotear a Combate, ya no está en el aire, mejor q sean leones y cobras como el origen!!\', \'Vamos combatientes con todos y contra todos, ustedes pueden ❤️💚❤️💚❤️💚\', \'La verdad empiecen votando a Rosangela\'],[\'¡Ridículo argumentar lo del Estado Azteca! Por favor uno tendría que ser muy tonto para usar eso y decir que AMLO pacto con Peña Nieto. ¡Mentiras! Las llamadas de miedo siguen, el zócalo no se lo quisieron dar, y por Dios se gastan millones en mitines gubernamentales y AMLO cierre de campaña presidencial, están quejándose porque el azteca cobra medio millón de peso

In [13]:
doc = nlp(all_words_str)
for ent in doc.ents:
    print(ent.text, ent.label_)

ElTrollino YT_channel
German YT_channel
German YT_channel
Doc Tops YT_channel
TheWillyrex YT_channel
Campechaneando YT_channel
German YT_channel
Campechaneando YT_channel
Teleantioquia YT_channel
TheWillyrex YT_channel
Campechaneando YT_channel
EEG YT_channel
Campechaneando YT_channel
Campechaneando YT_channel
Campechaneando YT_channel
Doc Tops YT_channel
Campechaneando YT_channel
German YT_channel
JuegaGerman YT_channel
Katia Nabil YT_channel
German YT_channel
German YT_channel
German YT_channel
La Sombra del Imperio YT_channel
German YT_channel
German YT_channel
German YT_channel
JuegaGerman YT_channel
German YT_channel


In [14]:
from spacy import displacy
displacy.render(doc, style='ent')

In [14]:
df_recomend = (df['pushblishDayName'].value_counts())
weekdays = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_recomend = df_recomend.reindex(weekdays)
df_recomend

Monday       34
Tuesday      47
Wednesday    44
Thursday     28
Friday       51
Saturday     34
Sunday       12
Name: pushblishDayName, dtype: int64

In [15]:
nlp1 = spacy.load('es_core_news_md')
nlp1.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1df875e8580>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x1df875e99c0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1df873422d0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1df875e94e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1df805a6180>),
 ('lemmatizer', <spacy.lang.es.lemmatizer.SpanishLemmatizer at 0x1df876bd7c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1df873421f0>)]

In [16]:
nlp1.component_names

['tok2vec',
 'morphologizer',
 'parser',
 'senter',
 'attribute_ruler',
 'lemmatizer',
 'ner']

In [17]:
# add textblob to spacy pipeline
nlp1.add_pipe('spacytextblob')

In [18]:
nlp1.pipe_names

['tok2vec',
 'morphologizer',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'spacytextblob']

In [63]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp2 = spacy.load('es_core_news_md')
sample1 = pd.read_csv('C:/Users/qsavy_1/CCD/ccd_meta/kaggle/comments_sample_42.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qsavy_1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
sample1

,Unnamed: 0,video_id,comments,comments_no_stopwords
0,65144,t0OzUTYAYSg,"['Súper canelo', 'Súper', 'CANELO A DE SER DEL...",NaN
1,39168,snzdubLe504,['Realmente todo aquello que me molesta del ot...,NaN
2,55151,3Yfnx1U7mvE,['antes de que lo vea un medico laboral de col...,NaN
3,59761,v1aYweBdEoA,"['Título de mrd', 'Pumas equipo chicoo']",NaN
4,44076,WLZl3XO-jeI,['Ya era hora que saliera del aire ese program...,NaN
...,...,...,...,...
6622,33999,Y8pjyyQg0O8,[],NaN
6623,15176,QimFFn81RUc,"['MILTON CARAGLIO ES UNA ESTAFA, SÍ, UNA ESTAF...",NaN
6624,63692,1wDSNar5Nm0,[],NaN
6625,45292,mJ_aytTDcdQ,"['sienpre sersan mejores los conbatiente', 'Qu...",NaN


In [65]:
corpus_spacy = []
for i in sample1['comments']:
    review = re.sub('[^a-zA-z]',' ',i)
    review = review.lower()
    stwords = stopwords.words('spanish')
    #stwords.remove("not")
    review = nlp1(review)
    review = [token.lemma_ for token in review if str(token) not in stwords]
    review = ' '.join(review)
    corpus_spacy.append(review)
print(corpus_spacy)

['[ s per canelo     s per     canelo ser pri solo saber robar hacer teatro    lamentable q seguir transmitir     hacer rato aventar tiro aqu   barrio m s vergazos            boxeo decadente    ]', '[ realmente aquel molestar   justo propio sombra    d molestir pues n vencer ego    dif cil trabajo   or raz n saber comprender realmente decir   emoción barca mar    tocar   vivir planeta vencer él      pregunta     quitar defecto personalidad    forma   m desintegrar interior malo     si dar comer perrito identificar exterior   perturbar   dar pauta identificir tambien llevar defecto   logico estar decir hablar hablar quuedar igual   lamentable    mas importante decir   mantener confusion   nivel mundial    mas embargo gnosis si decir elimnar defecto   liberar defecto   quedar resultado   conciencia recuperado   dicho conciencia atrapado defecto   mayor eliminacion defecto mayor conciencia    ir curso comprender metodo eliminacion defecto   liberacion conciencia      acuerdo or emoción   

In [66]:
print(len(corpus_spacy))

6627


In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [68]:
cv = CountVectorizer()
X = cv.fit_transform(corpus_spacy).toarray()
y = cv.fit_transform(sample.iloc[:,1]).toarray()
#y = sample1.iloc[:,1].values

In [69]:
X, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64),
 array(['t0OzUTYAYSg', 'snzdubLe504', '3Yfnx1U7mvE', ..., '1wDSNar5Nm0',
        'mJ_aytTDcdQ', 'cPZT6Kq6DZ8'], dtype=object))

In [70]:
cv.get_feature_names()

C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['a_hf',
 'aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaaa',
 'aaaaaaaaa',
 'aaaaaaaaaa',
 'aaaaaaaaaaa',
 'aaaaaaaaaaaa',
 'aaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaakjcdcfghsdgytnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

In [71]:

len(cv.get_feature_names())

38825

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [73]:
#MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred, average='macro')
score3= recall_score(y_test,y_pred, average='macro')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Accuracy is  0.08 %
Precision is  0.0
Recall is  0.0


C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
#BernoulliNB
classifier = BernoulliNB(alpha=0.1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred, average='weighted')
score3= recall_score(y_test,y_pred, average='weighted')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  1]
 [ 0  0  0 ...  0  0 11]]


Accuracy is  8.27 %
Precision is  0.01
Recall is  0.08


C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
#LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred, average='weighted')
score3= recall_score(y_test,y_pred, average='weighted')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0 11]]


Accuracy is  8.27 %
Precision is  0.01
Recall is  0.08


C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\qsavy_1\.conda\envs\scraper\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0

In [33]:
classifier.coef_

array([[-3.90751771e-04, -6.35154726e-04, -1.72397323e-03, ...,
        -8.59405148e-04, -2.35234615e-03, -6.03757490e-02],
       [-3.08108008e-05, -4.93388707e-05, -3.45057487e-04, ...,
        -7.48092551e-05, -1.22782654e-04, -1.22988671e-02],
       [-7.34950231e-05, -4.83066334e-04, -5.81381759e-04, ...,
        -1.82382525e-04, -3.51650828e-04, -1.92748794e-02],
       ...,
       [-3.50238938e-04, -3.33175895e-04, -1.92490291e-03, ...,
        -5.31219369e-04, -1.42843384e-03, -6.00943905e-02],
       [-2.38321951e-04, -3.47147817e-04, -1.40804920e-03, ...,
        -5.27365548e-04, -1.37411238e-03, -5.15007212e-02],
       [-3.55562303e-03, -2.55759176e-04, -3.85356349e-03, ...,
        -5.79098430e-03, -1.57006916e-01, -5.30366413e-01]])

In [34]:
classifier.classes_

array(['["El ladrón cayo con las cuatro gomas pa\' arriba. Jijijijijijijijijijiji🤑", \'❤❤❤❤\', \'Jajaja, a buebo!\']',
       '["If you are thinking to appeal to goodness infinity of Iraam. DON\'T!!! This is the Trailer -> https://www.youtube.com/watch?v=lttrIPDFplU", \'Baby Driver fue la razón por la que empecé a ver Cinerds (cuando me la recomendaron muchas personas) y yo con el trailer creí que sería ooootra película de acción y autos, pero me dijeron "como crees? si la dirige Edgar Wright" a lo que contesté "¿Quién?". Y aquí estoy.\', \'¿qué pedo? \\nYa soy su faaaaaan. \\nMe acabo de enamorar de su canal Chumelitos.\', \'quien vergas es Hiram?\', \'Chingón el especial!\', \'Vik, Oswaldo, Riky.... En general gracias  a todo el equipo de máquina por crear contenido chingón! !!\', \'Dejen el soundtrack de atomic Blonde\', \'Rifense un especial de Wes Anderson, por favor. Si ya lo hicieron, podrían ser tan amables de dejarme el link. Gracias por su atención.\', \'Solo les voy a dar la